# Weak-lensing galaxy shape catalogue validation

## Maps

Contents
- Create convergence maps

> **_NOTE:_** Before running this notebook, set kernel to `main_set.ipynb'

In [ ]:
from scipy.ndimage import gaussian_filter

from lenspack.utils import bin2d
from lenspack.image.inversion import ks93

In [ ]:
from sp_validation.survey import *
from sp_validation.util import *
from sp_validation.basic import *
from sp_validation.plots import *
from sp_validation.cosmology import *

# Pixelise ellipticities

In [ ]:
# Compute number of pixels
Nx = {}
Ny = {}
for sh in shapes:
    Nx[sh] = int(size_x_deg[sh] / pixel_size_emap_amin * 60)
    Ny[sh] = int(size_y_deg[sh] / pixel_size_emap_amin * 60)
    print_stats(f'{sh}:', stats_file, verbose=verbose)
    print_stats(
        f'Numbers of ellipticity pixels for KS93 = ({Nx[sh]}, {Ny[sh]})',
        stats_file,
        verbose=verbose
    )

In [ ]:
# Bin in 2D

g_corr_mc_map = {}

for sh in shapes:
    g1_tmp, g2_tmp = bin2d(
        x[sh],
        y[sh],
        npix=(Nx[sh], Ny[sh]), 
        v=(g_corr_mc[sh][0], g_corr_mc[sh][1]),
        extent=(min_x[sh], max_x[sh], min_y[sh], max_y[sh])
    )

    g_corr_mc_map[sh] = np.array([g1_tmp, g2_tmp])

# Create convergence maps

In [ ]:
# Transform gamma -> kappa using the Kaiser-Squires (1993) algorithm

kappaE = {}
kappaB = {}

for sh in shapes:
    kappaE[sh], kappaB[sh] = ks93(
        g1_sign * g_corr_mc_map[sh][0],
        g2_sign * g_corr_mc_map[sh][1]
    )

In [ ]:
# Smooth with Gaussian filter

kappaE_sm = {}
kappaB_sm = {}

for sh in shapes:
    kappaE_sm[sh] = gaussian_filter(kappaE[sh], smoothing_scale_pix)
    kappaB_sm[sh] = gaussian_filter(kappaB[sh], smoothing_scale_pix)

In [ ]:
# Get cluster information
cluster_cat_name = 'HFI_PCCS_SZ-union_R2.08.fits.gz'
vos_dir = 'vos:cfis/cosmostat/cosmology/external/Planck'

clusters = get_clusters(cluster_cat_name, vos_dir, data_dir, name, verbose=verbose)

print_stats(
    f"{len(clusters['ra'])} clusters found in {name} footprint",
    stats_file,
    verbose=verbose
)

In [ ]:
# Project cluster positions

# Get first shape method, use the same for all projections
for sh in shapes:
    sh0 = sh
    break

if wrap_ra != 0:
    clusters['ra_wrap'] = (clusters['ra'] + wrap_ra) % 360 - wrap_ra + 360
else:
    clusters['ra_wrap'] = clusters['ra']

x_cluster, y_cluster =  radec2xy(
    ra_mean[sh0],
    dec_mean[sh0],
    clusters['ra_wrap'],
    clusters['dec']
)
clusters['x'] = x_cluster
clusters['y'] = y_cluster

# Plot maps

## Convergence maps

### With Planck clusters and grid

In [ ]:
vlim = {}

for sh in shapes:

    title = f'{sh} $\kappa_{{\\rm E}}$'
    out_path = f'{plot_dir}/kappa_E_{sh}+clusters.png'

    vlim[sh] = plot_map(
        kappaE_sm[sh],
        ra_wrap[sh],
        dec[sh],
        min_x[sh],
        max_x[sh],
        min_y[sh],
        max_y[sh],
        Nx[sh],
        Ny[sh],
        title,
        out_path,
        clusters=clusters
    )

In [ ]:
for sh in shapes:
    
    title = f'{sh} $\kappa_{{\\rm B}}$'
    out_path = f'{plot_dir}/kappa_B_{sh}+clusters.png'

    plot_map(
        kappaB_sm[sh],
        ra_wrap[sh],
        dec[sh],
        min_x[sh],
        max_x[sh],
        min_y[sh],
        max_y[sh],
        Nx[sh],
        Ny[sh],
        title,
        out_path,
        vlim=vlim[sh],
        clusters=clusters
    )

In [ ]:
### Without grid lines

In [ ]:
vlim = {}

for sh in shapes:

    title = f'{sh} $\kappa_{{\\rm E}}$'
    out_path = f'{plot_dir}/kappa_E_{sh}_nogrid.png'

    vlim[sh] = plot_map(
        kappaE_sm[sh],
        ra_wrap[sh],
        dec[sh],
        min_x[sh],
        max_x[sh],
        min_y[sh],
        max_y[sh],
        Nx[sh],
        Ny[sh],
        title,
        out_path,
        grid=False
    )

In [ ]:
for sh in shapes:
    
    title = f'{sh} $\kappa_{{\\rm B}}$'
    out_path = f'{plot_dir}/kappa_B_{sh}_nogrid.png'

    plot_map(
        kappaB_sm[sh],
        ra_wrap[sh],
        dec[sh],
        min_x[sh],
        max_x[sh],
        min_y[sh],
        max_y[sh],
        Nx[sh],
        Ny[sh],
        title,
        out_path,
        vlim=vlim[sh],
        grid=False
    )

### At cut-out coordinates

In [ ]:
if map_cut_coords:
    for sh in shapes:
    
        title = ''
        out_path = f'{plot_dir}/kappa_E_{sh}_nogrid_cutout.png'

        plot_map(
            kappaE_sm[sh],
            ra_wrap[sh],
            dec[sh],
            min_x[sh],
            max_x[sh],
            min_y[sh],
            max_y[sh],
            Nx[sh],
            Ny[sh],
            title,
            out_path,
            vlim=vlim[sh],
            grid=False,
            map_cut_coords=map_cut_coords,
            dpi=600,
            colorbar=False
        )

## Stacked convergence maps

In [ ]:
if len(clusters['ra']) > 1:

    # Comoving separation around cluster centers, at cluster redshift, in [Mpc]
    radius = 5

    res_stack_mm = {}

    # Stack galaxies
    for sh in shapes:
        res_stack_mm[sh] = stack_mm3(
            ra_wrap[sh],
            dec[sh],
            g_corr_mc[sh][0],
            g_corr_mc[sh][1],
            w[sh],
            clusters['ra_wrap'],
            clusters['dec'],
            clusters['z'],
            radius=radius,
            n_match=1000000
        )

In [ ]:
# Plot stacked galaxy density, to check how uniform distribution is. Sometimes at the edges the number
# of galaxies drops visibly

if len(clusters['ra']) > 1:

    for sh in shapes:
        plt.figure(figsize=(10, 10))
        plt.hexbin(
            res_stack_mm[sh][0],
            res_stack_mm[sh][1],
            gridsize=100,
            cmap='gist_stern'
        )
        cbar = plt.colorbar()
        cbar.set_label('Number count', rotation=270)
        plt.title(f'{sh} stacked galaxy density plot')

In [ ]:
# Bin stacked ellipticities

if len(clusters['ra']) > 1:

    e1map_stack = {}
    e2map_stack = {}
    npix = 2048

    for sh in shapes:
        e1map_stack[sh], e2map_stack[sh] = bin2d(
            res_stack_mm[sh][0],
            res_stack_mm[sh][1],
            v=(res_stack_mm[sh][2], -res_stack_mm[sh][3]),
            w=res_stack_mm[sh][4], 
            npix=npix
    )

In [ ]:
if len(clusters['ra']) > 1:

    kappaE_stack = {}
    kappaB_stack = {}
    kappaE_stack_sm = {}
    kappaB_stack_sm = {}
 
    for sh in shapes:
        # transform to gamma -> kappa via the aisers & Squires (1993) algorithm
        kappaE_stack[sh], kappaB_stack[sh] = ks93(e1map_stack[sh], e2map_stack[sh])
    
        # Smooth
        kappaE_stack_sm[sh] = gaussian_filter(kappaE_stack[sh], smoothing_scale_pix)
        kappaB_stack_sm[sh] = gaussian_filter(kappaB_stack[sh], smoothing_scale_pix)

In [ ]:
if len(clusters['ra']) > 1:

    for sh in shapes:
        title = f'{sh} kappa_E'
        output_path = f'{plot_dir}/kappaE_stacked_{sh}.png'

        vlim[sh] = plot_map_stacked(kappaE_stack_sm[sh], title, radius, output_path)

In [ ]:
if len(clusters['ra']) > 1:

    for sh in shapes:
        title = f'{sh} kappa_B'
        output_path = f'{plot_dir}/kappaB_stacked_{sh}.png'

        plot_map_stacked(kappaB_stack_sm[sh], title, radius, output_path, vlim=vlim[sh])